## Resumen diario de KPIs con ayuda de Gemini

Python 3.9.6

> pip install google-genai pandas


In [1]:
pip install google-genai pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_genai-1.46.0-py3-none-any.whl.metadata (46 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
Using cached google_genai-1.46.0-py3-none-any.whl (239 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 1.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from google import genai
from google.genai import types
import pandas as pd
import json
from pathlib import Path

# Cargar y preparar datos
CSV_PATH = "data/coffee-sales.csv"

df = pd.read_csv(
    CSV_PATH,
    parse_dates=["date", "datetime"],
    dtype={
        "cash_type": "string",
        "card": "string",
        "money": "float64",
        "coffee_name": "string",
    }
)

# Normalizaciones simples
df["cash_type"] = df["cash_type"].str.lower().fillna("card")
df["coffee_name"] = df["coffee_name"].str.strip()

# Filtra registros con monto válido
df = df[df["money"].notna()]

# KPIs por tipo de café
agg = df.groupby("coffee_name").agg(
    revenue=("money", "sum"),
    orders=("money", "count"),
    avg_ticket=("money", "mean"),
).reset_index()

# Share de pago por café (cash/card)
mix = (
    df.assign(cnt=1)
      .pivot_table(index="coffee_name", columns="cash_type", values="cnt", aggfunc="sum", fill_value=0)
      .rename_axis(None, axis=1)
      .reset_index()
)

# Unir y calcular proporciones
kpis = agg.merge(mix, on="coffee_name", how="left")
if "cash" not in kpis.columns: kpis["cash"] = 0
if "card" not in kpis.columns: kpis["card"] = 0

kpis["cash_share"] = (kpis["cash"] / (kpis["cash"] + kpis["card"])).fillna(0)
kpis["card_share"] = (kpis["card"] / (kpis["cash"] + kpis["card"])).fillna(0)

# Ordena por revenue desc y toma top (N)
kpis = kpis.sort_values("revenue", ascending=False)
top = kpis.head(20)  # Ajusta si quieres todos, cuidado con tus tokens

# Compacta para el prompt (redondea para legibilidad)
def r(x): 
    return float(round(x, 2))

sample = []
for _, row in top.iterrows():
    sample.append({
        "coffee_name": row["coffee_name"],
        "revenue": r(row["revenue"]),
        "orders": int(row["orders"]),
        "avg_ticket": r(row["avg_ticket"]),
        "cash_share": r(row["cash_share"]),
        "card_share": r(row["card_share"]),
    })

# También puedes incluir una visión por día si lo necesitas:
date_span = f"{df['date'].min().date()} → {df['date'].max().date()}"

# Prompt y llamada a Gemini
client = genai.Client()               # Usa GEMINI_API_KEY del entorno
MODEL = "gemini-2.0-flash"            # Rápido y suficiente para este caso

prompt = f"""
Actúa como analista de negocio del coffee shop.
Período analizado: {date_span}

A partir de este resumen JSON por tipo de café:
{json.dumps(sample, ensure_ascii=False)}

Devuélveme en un formato markdown:
1) 3 hallazgos clave (con cifras específicas)
2) 2 riesgos o anomalías potenciales (con hipótesis para validar)
3) 3 acciones concretas para mañana (dueño sugerido y métrica a monitorear)
Usa un tono ejecutivo y conciso (máx. 150 palabras).
"""

resp = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config=types.GenerateContentConfig(temperature=0.3),  # más preciso/estable
)

# 0.0-0.3: más conservador
# 0.4-0.7: equilibrio
# 0.8-1: más "creativo" o variable

summary_md = resp.text
print(summary_md)

# Guardar salida
Path("output").mkdir(parents=True, exist_ok=True) # En caso de no existir, crea el directorio

out_path = Path("output/coffee-resumen.md")
out_path.write_text(summary_md, encoding="utf-8")

print(f"OK: resumen guardado en {out_path.resolve()}")

## Análisis de Desempeño del Coffee Shop (2024-03-01 → 2025-03-23)

**1) Hallazgos Clave:**

*   **Liderazgo en Ingresos:** El Latte genera la mayor parte de los ingresos con $27,866.3, impulsado por 782 órdenes.
*   **Popularidad del Americano:** El Americano con Leche es el producto con más órdenes (824), generando $25,269.12 en ingresos.
*   **Ticket Promedio Elevado:** Cappuccino y Hot Chocolate presentan un ticket promedio alto ($36.0 y $36.07 respectivamente), indicando potencial para aumentar ventas cruzadas.

**2) Riesgos/Anomalías Potenciales:**

*   **Bajo Desempeño del Espresso:** Con solo $2,814.28 en ingresos y 134 órdenes, el Espresso podría estar subestimado. *Hipótesis: Baja visibilidad en el menú o percepción de precio elevado.*
*   **Dependencia de Pagos con Tarjeta:** La alta proporción de pagos con tarjeta (96-98%) podría generar costos de transacción significativos. *Hipótesis: Falta de incentivos para el uso de efectivo o comodidad percibida por los clientes.*

**